# Análise de Dados de Feriados e Clima para 2024

Este notebook realiza uma análise dos feriados e do clima no Rio de Janeiro para o ano de 2024. Foi utilizado dados de APIs para obter informações sobre feriados e clima para responder a perguntas especificadas.

1. **Quantos feriados há no Brasil em todo o ano de 2024?**

2. **Qual mês de 2024 tem o maior número de feriados?**

3. **Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?**

4. **Qual foi a temperatura média em cada mês?**  
   Utilize a Open-Meteo Historical Weather API para obter as temperaturas médias diárias no Rio de Janeiro de 01/01/2024 a 01/08/2024.  
   
5. **Qual foi o tempo predominante em cada mês nesse período?**  
   Utilize como referência para o código de tempo (_weather_code_) o seguinte link: [WMO Code](https://gist.github.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c).

6. **Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?**

7. **Considere as seguintes suposições:**
   - O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
   - Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
   - O cidadão carioca só vai à praia quando não está com frio;
   - O cidadão carioca também só vai à praia em dias com sol, evitando dias **totalmente** nublados ou chuvosos (considere _weather_code_ para determinar as condições climáticas).

   Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?

8. **Qual foi o feriado "mais aproveitável" de 2024?**  
   Considere o melhor par tempo e temperatura.

# Importação de Bibliotecas

In [1]:
import requests
import pandas as pd
import json
import os

# Funções Auxiliares

Algumas funções auxiliares para facilitar o processamento dos dados.

In [2]:
# Funções Auxiliares
def get_response(url):
    """
    Faz uma requisição GET para a URL fornecida e retorna a resposta em formato JSON.

    Parâmetros:
    url (str): A URL para a qual a requisição será enviada.

    Retorna:
    dict ou None: Retorna o conteúdo JSON da resposta se a requisição for bem-sucedida,
    ou None se houver um erro.
    """
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao fazer a requisição: {response.status_code}")
        return None


def process_holidays_data(response_holidays):
    """
    Processa os dados dos feriados convertendo-os para um DataFrame.

    Parâmetros:
    response_holidays (dict): A resposta JSON contendo os dados dos feriados.

    Retorna:
    pandas.DataFrame: Um DataFrame com os dados dos feriados processados,
    incluindo uma coluna com o nome do mês.
    """
    df_holidays = pd.DataFrame(response_holidays)
    df_holidays["date"] = pd.to_datetime(df_holidays["date"])
    df_holidays["month"] = df_holidays["date"].dt.month_name()
    return df_holidays


def process_weather_data(response_weather):
    """
    Processa os dados do clima, combinando informações horárias e diárias em um único DataFrame.

    Parâmetros:
    response_weather (dict): A resposta JSON contendo os dados climáticos.

    Retorna:
    pandas.DataFrame: Um DataFrame com os dados climáticos processados,
    incluindo colunas para data, temperatura e código de clima.
    """
    df_hourly_temperatures = pd.DataFrame(response_weather["hourly"])
    df_hourly_temperatures["date"] = pd.to_datetime(
        df_hourly_temperatures["time"]
    ).dt.date

    df_daily_weather = pd.DataFrame(response_weather["daily"])
    df_daily_weather["date"] = pd.to_datetime(df_daily_weather["time"]).dt.date

    df_merged = df_hourly_temperatures.merge(df_daily_weather, on="date", how="outer")
    df_merged = df_merged[["time_x", "temperature_2m", "weather_code"]]

    df_merged.columns = ["datetime", "temperature", "weather_code"]
    df_merged["datetime"] = pd.to_datetime(df_merged["datetime"])

    return df_merged


def read_json_file(file_path):
    """
    Lê o conteúdo de um arquivo JSON e retorna os dados.

    Parâmetros:
    file_path (str): O caminho do arquivo JSON.

    Retorna:
    dict: O conteúdo JSON do arquivo.
    """
    with open(file_path, "r") as f:
        return json.load(f)


def get_weather_code_df():
    """
    Obtém o DataFrame contendo os códigos de clima a partir de um arquivo JSON.

    Retorna:
    pandas.DataFrame ou None: Um DataFrame com os códigos de clima e suas descrições
    em português e inglês, ou None se o arquivo não for encontrado.
    """
    file = "../data/weather_code.json"
    if not file in os.listdir():
        return None

    weather_codes_json = read_json_file(file)
    df_weather_codes = pd.DataFrame(weather_codes_json).T
    df_weather_codes["weather_day"] = df_weather_codes["day"].apply(
        lambda x: x["description"]
    )
    df_weather_codes["weather_night"] = df_weather_codes["night"].apply(
        lambda x: x["description"]
    )
    df_weather_codes.reset_index(inplace=True)
    df_weather_codes = df_weather_codes[["index", "weather_day", "weather_night"]]
    df_weather_codes.columns = ["weather_code", "weather_day", "weather_night"]
    df_weather_codes["weather_code"] = df_weather_codes["weather_code"].astype(int)

    # Traduzindo o nome do clima para o português
    weather_translation = {
        "Sunny": "Ensolarado",
        "Clear": "Limpo",
        "Mainly Sunny": "Principalmente Ensolarado",
        "Mainly Clear": "Principalmente Limpo",
        "Partly Cloudy": "Parcialmente Nublado",
        "Cloudy": "Nublado",
        "Foggy": "Nebuloso",
        "Rime Fog": "Névoa de Gelo",
        "Light Drizzle": "Chuvisco Leve",
        "Drizzle": "Chuvisco",
        "Heavy Drizzle": "Chuvisco Forte",
        "Light Freezing Drizzle": "Chuvisco Leve Congelante",
        "Freezing Drizzle": "Chuvisco Congelante",
        "Light Rain": "Chuva Leve",
        "Rain": "Chuva",
        "Heavy Rain": "Chuva Forte",
        "Light Freezing Rain": "Chuva Leve Congelante",
        "Freezing Rain": "Chuva Congelante",
        "Light Snow": "Neve Leve",
        "Snow": "Neve",
        "Heavy Snow": "Neve Forte",
        "Snow Grains": "Grãos de Neve",
        "Light Showers": "Chuvas Leves",
        "Showers": "Chuvas",
        "Heavy Showers": "Chuvas Fortes",
        "Light Snow Showers": "Chuvas de Neve Leves",
        "Snow Showers": "Chuvas de Neve",
        "Thunderstorm": "Tempestade",
        "Light Thunderstorms With Hail": "Tempestades Leves com Granizo",
        "Thunderstorm With Hail": "Tempestade com Granizo",
    }
    df_weather_codes["weather_day_pt"] = df_weather_codes["weather_day"].map(
        weather_translation
    )
    df_weather_codes["weather_night_pt"] = df_weather_codes["weather_night"].map(
        weather_translation
    )

    return df_weather_codes


def display_holiday_details(df):
    """
    Exibe os detalhes dos feriados contidos em um DataFrame.

    Parâmetros:
    df (pandas.DataFrame): O DataFrame contendo os dados dos feriados.
    """
    for i, row in df.iterrows():
        print(
            f"""Dia: {row['date'].date()} - Feriado \"{row['localName']}\"
Temperatura média: {row['temperatura_media']}
Clima durante o dia: {row['weather_day_pt']}
Clima durante a noite: {row['weather_night_pt']}
"""
        )

# Carregamento dos Dados

Carregando os dados de feriados e clima a partir das APIs e arquivos.

In [3]:
# carregando os dados através das APIs e arquivos

url_holidays = "https://date.nager.at/api/v3/publicholidays/2024/BR"
url_weather = "https://archive-api.open-meteo.com/v1/archive?latitude=-22.9064&longitude=-43.1822&start_date=2024-01-01&end_date=2024-08-01&hourly=temperature_2m&daily=weather_code&timezone=America%2FSao_Paulo"

response_holidays = get_response(url_holidays)
df_feriados = process_holidays_data(response_holidays)

response_weather = get_response(url_weather)
df_temperaturas_rj = process_weather_data(response_weather)

df_codigos_clima = get_weather_code_df()

# 1. Quantos feriados há no Brasil em todo o ano de 2024?

Para responder a essa pergunta, vamos contar o número total de registros no DataFrame que contém os feriados no Brasil para o ano de 2024.


In [4]:
print("Quantidade de feriados em 2024:", df_feriados.shape[0])

Quantidade de feriados em 2024: 14


# 2. Qual mês de 2024 tem o maior número de feriados?

Para responder a essa pergunta, vamos agrupar os feriados por mês e identificar qual mês possui a maior quantidade de feriados.


In [5]:
df_feriados_por_mes = df_feriados["mes"].value_counts()
df_feriados_por_mes
max_feriados = df_feriados_por_mes.max()
meses_mais_feriados = df_feriados_por_mes[df_feriados_por_mes == max_feriados]
print(
    f"Mês(es) de 2024 com o maior número de feriados: {meses_mais_feriados.index.to_list()}, com {meses_mais_feriados.iloc[0]} feriados em cada"
)

Mês(es) de 2024 com o maior número de feriados: ['February', 'March', 'May', 'November'], com 2 feriados em cada


# 3. Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?

Para responder a essa pergunta, vamos criar uma nova coluna distinguindo o dia da semana partindo da data, identificar os feriados que caem em dias úteis (segunda a sexta-feira) e contar o número total desses feriados.


In [6]:
df_feriados["dia_da_semana"] = df_feriados["date"].dt.dayofweek
n_feriados_dia_util = df_feriados[df_feriados["dia_da_semana"] < 5].shape[0]

print(f"Número de feriados em dias de semana: {n_feriados_dia_util}")

Número de feriados em dias de semana: 9


# 4. Qual foi a temperatura média em cada mês?

Para responder a essa pergunta, vamos criar uma coluna 'ano_mes' para agrupar as temperaturas por mês e, em seguida, calcular a média de temperatura para cada um desses meses.


In [7]:
df_temperaturas_rj["ano_mes"] = df_temperaturas_rj["datetime"].dt.to_period("M")

temperatura_media_mensal = (
    df_temperaturas_rj.groupby("ano_mes")["temperature"].mean().reset_index()
)
print("Temperatura média por mês:")
temperatura_media_mensal

Temperatura média por mês:


,ano_mes,temperature
0,2024-01,26.677419
1,2024-02,27.198420
2,2024-03,26.400538
3,2024-04,25.154722
4,2024-05,24.975000
5,2024-06,22.634306
6,2024-07,21.089785
7,2024-08,21.216667


## 5: Qual foi o tempo predominante em cada mês nesse período?

É criado uma coluna Ano / Mes para possibilitar a agregação e calcular valores que mais se repetem.

In [9]:
df_temperaturas_rj["ano_mes"] = df_temperaturas_rj["datetime"].dt.to_period("M")
tempo_predominante_mensal = (
    df_temperaturas_rj.groupby("ano_mes")["weather_code"]
    .agg(lambda x: x.mode()[0])
    .reset_index()
)
tempo_predominante_mensal = tempo_predominante_mensal.merge(
    df_codigos_clima, on="weather_code"
)

print("Tempo predominante por mês:")
tempo_predominante_mensal

Tempo predominante por mês:


,ano_mes,weather_code,weather_day,weather_night,weather_day_pt,weather_night_pt
0,2024-01,63,Rain,Rain,Chuva,Chuva
1,2024-02,51,Light Drizzle,Light Drizzle,Chuvisco Leve,Chuvisco Leve
2,2024-03,51,Light Drizzle,Light Drizzle,Chuvisco Leve,Chuvisco Leve
3,2024-04,3,Cloudy,Cloudy,Nublado,Nublado
4,2024-05,51,Light Drizzle,Light Drizzle,Chuvisco Leve,Chuvisco Leve
5,2024-06,1,Mainly Sunny,Mainly Clear,Principalmente Ensolarado,Principalmente Limpo
6,2024-07,3,Cloudy,Cloudy,Nublado,Nublado
7,2024-08,2,Partly Cloudy,Partly Cloudy,Parcialmente Nublado,Parcialmente Nublado


# 6. Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?

Para responder a essa pergunta, vamos identificar os feriados no período especificado, calcular a temperatura média e associar o clima predominante (código de tempo) em cada um desses dias.


In [10]:
inicio = pd.to_datetime("2024-01-01")
fim = pd.to_datetime("2024-08-01")
datas_feriados_2024_filtrado = df_feriados[
    (df_feriados["date"] >= inicio) & (df_feriados["date"] <= fim)
]["date"]

df_temperaturas_rj["date"] = df_temperaturas_rj["datetime"]
# Filtra o DataFrame de temperaturas para incluir apenas as datas dos feriados
df_temperaturas_feriados = df_temperaturas_rj[
    df_temperaturas_rj["date"].isin(datas_feriados_2024_filtrado)
]

# Calcula a temperatura média para cada feriado
df_temperaturas_medias_feriados = (
    df_temperaturas_feriados.groupby("date")
    .agg(
        temperatura_media=("temperature", "mean"),
        weather_code=("weather_code", "first"),
    )
    .reset_index()
)

df_temperaturas_medias_feriados = df_temperaturas_medias_feriados.merge(
    df_codigos_clima, on="weather_code"
)
df_temperaturas_medias_feriados = df_temperaturas_medias_feriados.merge(
    right=df_feriados[["date", "localName"]], on="date"
)


exibir_detalhes_feriados(df_temperaturas_medias_feriados)

Dia: 2024-01-01 - Feriado "Confraternização Universal"
Temperatura média: 23.1
Clima durante o dia: Chuvisco Leve
Clima durante a noite: Chuvisco Leve

Dia: 2024-02-12 - Feriado "Carnaval"
Temperatura média: 27.4
Clima durante o dia: Principalmente Ensolarado
Clima durante a noite: Principalmente Limpo

Dia: 2024-02-13 - Feriado "Carnaval"
Temperatura média: 27.2
Clima durante o dia: Chuvisco Leve
Clima durante a noite: Chuvisco Leve

Dia: 2024-03-29 - Feriado "Sexta-feira Santa"
Temperatura média: 23.1
Clima durante o dia: Chuva Leve
Clima durante a noite: Chuva Leve

Dia: 2024-03-31 - Feriado "Domingo de Páscoa"
Temperatura média: 23.7
Clima durante o dia: Chuvisco
Clima durante a noite: Chuvisco

Dia: 2024-04-21 - Feriado "Dia de Tiradentes"
Temperatura média: 20.5
Clima durante o dia: Parcialmente Nublado
Clima durante a noite: Parcialmente Nublado

Dia: 2024-05-01 - Feriado "Dia do Trabalhador"
Temperatura média: 25.0
Clima durante o dia: Ensolarado
Clima durante a noite: Limpo

D

## 7: Considere as seguintes suposições:
   - O cidadão carioca considera "frio" um dia cuja temperatura média é menor que 20ºC;
   - Um feriado bem aproveitado no Rio de Janeiro é aquele em que se pode ir à praia;
   - O cidadão carioca só vai à praia quando não está com frio;
   - O cidadão carioca também só vai à praia em dias com sol, evitando dias **totalmente** nublados ou chuvosos (considere _weather_code_ para determinar as condições climáticas).

   Houve algum feriado "não aproveitável" em 2024? Se sim, qual(is)?

Para responder a essa pergunta, vamos definir critérios para considerar um feriado como "não aproveitável" (temperatura média abaixo de 20ºC ou clima desfavorável) e identificar quais feriados se enquadram nesses critérios.

In [11]:
condicoes = (df_temperaturas_medias_feriados["temperatura_media"] > 20) & (
    df_temperaturas_medias_feriados["weather_code"].isin([0, 1, 2])
)
feriados_aproveitaveis = df_temperaturas_medias_feriados[condicoes]
feriados_nao_aproveitaveis = df_temperaturas_medias_feriados[~condicoes]

print("Feriados não aproveitáveis para o carioca:")
display_holiday_details(feriados_nao_aproveitaveis)

Feriados não aproveitáveis para o carioca:
Dia: 2024-01-01 - Feriado "Confraternização Universal"
Temperatura média: 23.1
Clima durante o dia: Chuvisco Leve
Clima durante a noite: Chuvisco Leve

Dia: 2024-02-13 - Feriado "Carnaval"
Temperatura média: 27.2
Clima durante o dia: Chuvisco Leve
Clima durante a noite: Chuvisco Leve

Dia: 2024-03-29 - Feriado "Sexta-feira Santa"
Temperatura média: 23.1
Clima durante o dia: Chuva Leve
Clima durante a noite: Chuva Leve

Dia: 2024-03-31 - Feriado "Domingo de Páscoa"
Temperatura média: 23.7
Clima durante o dia: Chuvisco
Clima durante a noite: Chuvisco

Dia: 2024-05-30 - Feriado "Corpus Christi"
Temperatura média: 20.4
Clima durante o dia: Chuvisco Leve
Clima durante a noite: Chuvisco Leve

Dia: 2024-07-09 - Feriado "Revolução Constitucionalista de 1932"
Temperatura média: 21.1
Clima durante o dia: Chuvisco
Clima durante a noite: Chuvisco



# 8. Qual foi o feriado "mais aproveitável" de 2024?

Para responder a essa pergunta, vamos encontrar o feriado com o clima mais ensolarado e a maior temperatura média, considerando esse o "mais aproveitável".


In [12]:
feriado_mais_aproveitavel = (
    df_temperaturas_medias_feriados[
        df_temperaturas_medias_feriados["weather_code"] == 0
    ]
    .sort_values(by="temperatura_media", ascending=False)
    .iloc[0]
)

print(f"O feriado mais aproveitável para o carioca foi o \"{feriado_mais_aproveitavel["localName"]}\" com uma temperatura média de {feriado_mais_aproveitavel['temperatura_media']}ºC")

O feriado mais aproveitável para o carioca foi o "Dia do Trabalhador" com uma temperatura média de 25.0ºC
